In [1]:
from methods.face_det_init_cnn import create_mtcnn
import mmcv
mtcnn = create_mtcnn(device='cuda')


In [2]:
img = open("img.png", "rb").read()
image = mmcv.imfrombytes(img)
imgs = [image for _ in range(128)]

In [4]:
lst = 0
for i in range(9):
    
    import time
    tic = time.time()
    boxes_list, confidence_list = mtcnn.detect(imgs)
    toc = time.time()
    lst = toc - tic
    print(f"Experiment with {1<<i} images => batch inference time = {lst}")
    
    #print(toc - tic > 2 * lst)
    

Experiment with 1 images => batch inference time = 0.2615039348602295
Experiment with 2 images => batch inference time = 0.21907567977905273
Experiment with 4 images => batch inference time = 0.2267763614654541
Experiment with 8 images => batch inference time = 0.22220826148986816
Experiment with 16 images => batch inference time = 0.22706365585327148
Experiment with 32 images => batch inference time = 0.22394776344299316
Experiment with 64 images => batch inference time = 0.22923898696899414
Experiment with 128 images => batch inference time = 0.2241837978363037
Experiment with 256 images => batch inference time = 0.22914910316467285


# Check model logs

In [29]:
import re
path = "/home/jafar/PyCharmProjects/Upwork/Andrei-Torchserve/torchserve/logs"

import os

pth = os.path.join(path, "model_metrics.log")

logs = open(pth, "r").read()
pth = os.path.join(path, "model_log.log")

logs = logs + "\n" +  open(pth, "r").read()
keywords = [
    # "PreprocessingTimeForBatch", 
    "BatchSize", 
    # "InferenceTimeForBatch", 
    "HandlerTime",
    # "InternalInferenceTimeForBatch"
    ]
for keyword in keywords:
    pattern = f'{keyword}.\w+.\w+:(\d+.?\d*)'
    matches = re.findall(pattern, logs)
    matches = list(map(lambda x: float(x), matches))
    #print(matches)
    import numpy
    arr = numpy.array(matches)
    print(pattern.split(".")[0], arr.min(), arr.max(), arr.mean(), arr.std())

#!rm /home/jafar/PyCharmProjects/Upwork/Andrei-Torchserve/torchserve/logs -r

BatchSize 100.0 126.0 118.72222222222223 4.283545894711385
HandlerTime 1315.32 1944.68 1799.0698148148147 93.63854837917074


# Check coordinator on jupyter

In [1]:
import unittest
import time
from coordinator import MMdetHandler
class Test_Coordinator(unittest.TestCase):
    def __init__(self, methodName="coordinator"):

        class Metrics:
            def add_time(self, *args, **kwargs):
                self.log(*args, **kwargs)
            def add_metric(self, *args, **kwargs):
                self.log(*args, **kwargs)
            def log(self, *args, **kwargs):
                print("Logger.args= ", *args)
                print("Logger.kwargs= ", *args)    

        class Context:
            system_properties = {
                "gpu_id" : "0"
            }

            metrics = Metrics()
            def get_request_header(self, *args, **kwargs):
                return True


        #super().__init__(methodName)
        self.model_handler = MMdetHandler()
        self.model_handler.initialize(context=Context())
        self.handle = self.model_handler.handle
        self.context = Context()

    def test_batch(self, batch_sz=128, img_path = "../tests/influencer.png"):
        img_binary = open(img_path, "rb").read()
        json_req = {"body" : img_binary}
        batch = [json_req for _ in range(batch_sz)]
        print("Initializing")
        def handle_once():
            tic = time.time()
            self.handle(batch, self.context)
            toc = time.time()
            print("HandleTime", toc - tic)
        for _ in range(1):
            handle_once()
tst = Test_Coordinator()

STUFF IN THE DIRECTORY


In [3]:
tst.test_batch()

Initializing
Logger.args=  BatchSize 128 b7d49987-8663-4b04-863f-c52747bc38be Batches
Logger.kwargs=  BatchSize 128 b7d49987-8663-4b04-863f-c52747bc38be Batches
Logger.args=  PackageSize 3 KB 06f3aec3-01ee-46d3-9beb-e50df8506302
Logger.kwargs=  PackageSize 3 KB 06f3aec3-01ee-46d3-9beb-e50df8506302
Logger.args=  PackageSize 3 KB 3aa202b1-d7e2-440e-bfdf-067ed9865f21
Logger.kwargs=  PackageSize 3 KB 3aa202b1-d7e2-440e-bfdf-067ed9865f21
Logger.args=  PackageSize 3 KB b7dfc4e9-985c-40a9-a48c-ecb56be383e2
Logger.kwargs=  PackageSize 3 KB b7dfc4e9-985c-40a9-a48c-ecb56be383e2
Logger.args=  PackageSize 3 KB 82632f65-4686-446e-83fc-bed07b6ff87a
Logger.kwargs=  PackageSize 3 KB 82632f65-4686-446e-83fc-bed07b6ff87a
Logger.args=  PackageSize 3 KB b38060fc-4449-49bd-aa03-5bf94d384a42
Logger.kwargs=  PackageSize 3 KB b38060fc-4449-49bd-aa03-5bf94d384a42
Logger.args=  PackageSize 3 KB cf98db14-5e4a-4c17-8c9c-767b6ff6e84c
Logger.kwargs=  PackageSize 3 KB cf98db14-5e4a-4c17-8c9c-767b6ff6e84c
Logger.args